In [ ]:
# ==============================================================================
# PRE-COMPUTATION WITH CUSTOM AUGMENTATION
# ==============================================================================
# This script uses my custom fine-tuned augmentation strategy to
# create the final pre-computed feature set. It involved solving a optimization 
# problem that we want to increase the no. of clips in such a way that the mean  
# is closest to without augment max class clips while making sure the balanced  
# distribution has as less std deviation as possible. 
# As the clips are multi-lable, simply repeating clips might also lead to increase  
# in no.s of max class which we want to leave as is.
## THIS IS CUSTBAL2, the the DATASET IS MORE balanced than CUSTBAL1. Changes made 
## in custom balancing strategy.
# ==============================================================================

import pandas as pd
import numpy as np
import librosa
from pathlib import Path
import os
from tqdm.auto import tqdm
import joblib

tqdm.pandas()

from sklearn.model_selection import GroupShuffleSplit

import torch
from transformers import Wav2Vec2Processor, Wav2Vec2Model

In [5]:
!pip install --no-deps audiomentations
!pip install numpy_minmax numpy_rms python_stretch
from audiomentations import Compose, AddGaussianNoise, PitchShift

In [10]:
!ls ../input

clips			  fluencybank_labels.csv  SEP-28k_labels.csv
fluencybank_episodes.csv  SEP-28k_episodes.csv


In [ ]:
# Configuration 
CLIPS_DIR = Path("/kaggle/input/clips/stuttering-clips/clips")
LABEL_FILE = "/kaggle/input/SEP-28k_labels.csv"
OUTPUT_DIR = Path("./output_wav2vec_custom_augmented")
OUTPUT_DIR.mkdir(exist_ok=True)

MODEL_NAME = "facebook/wav2vec2-base-960h"
SAMPLING_RATE = 16000
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")
MAX_SEQ_LEN = 150
EMBEDDING_DIM = 768

# Label Definitions 
LABEL_COLS = ['Prolongation', 'Block', 'SoundRep', 'WordRep', 'Interjection']
ALL_LABELS = LABEL_COLS + ['NoStutter']

BATCH_SIZE = 32

CLEANED_DF_PATH = OUTPUT_DIR / "df_multilabel.parquet"
TRAIN_DATA_PATH = OUTPUT_DIR / "train_data.npz"
VAL_DATA_PATH = OUTPUT_DIR / "val_data.npz"
TEST_DATA_PATH = OUTPUT_DIR / "test_data.npz"

Using device: cuda


In [17]:
print("--- Preparing and Splitting Data... ---")
if os.path.exists(CLEANED_DF_PATH):
    df = pd.read_parquet(CLEANED_DF_PATH)
else:
    df = pd.read_csv(LABEL_FILE)
    df['Clip'] = df['Show'].astype(str) + '_' + df['EpId'].astype(str) + '_' + df['ClipId'].astype(str)
    clips_in_folder = {c.stem for c in CLIPS_DIR.glob("*.wav")}
    df = df[df['Clip'].isin(clips_in_folder)].copy()
    def get_duration(clip_name):
        try: return librosa.get_duration(path=CLIPS_DIR / f"{clip_name}.wav")
        except Exception: return None
    df['duration'] = df['Clip'].progress_apply(get_duration)
    df.dropna(subset=['duration'], inplace=True)
    df = df[(df['duration'] > 2.95) & (df['duration'] < 3.05)].copy()
    for col in LABEL_COLS:
        df[col] = (df[col] > 0).astype(int)
    df['NoStutter'] = (df[LABEL_COLS].sum(axis=1) == 0).astype(int)
    df.to_parquet(CLEANED_DF_PATH)

df['speaker_id'] = df['Show']
gss_test = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
train_val_idx, test_idx = next(gss_test.split(df, groups=df['speaker_id']))
train_val_df = df.iloc[train_val_idx]
test_df = df.iloc[test_idx]
gss_val = GroupShuffleSplit(n_splits=1, test_size=0.15, random_state=42)
train_idx, val_idx = next(gss_val.split(train_val_df, groups=train_val_df['speaker_id']))
train_df = train_val_df.iloc[train_idx]
val_df = train_val_df.iloc[val_idx]
print("Data splitting complete.")

--- Preparing and Splitting Data... ---


  0%|          | 0/28177 [00:00<?, ?it/s]

Data splitting complete.


In [18]:
# distribution BEFORE augmentation 
print("\n--- Original Label Distribution in Training Set (Before Augmentation) ---")
print(train_df[ALL_LABELS].sum().sort_values(ascending=False))


--- Original Label Distribution in Training Set (Before Augmentation) ---
Block           7998
Interjection    5909
Prolongation    5569
NoStutter       4162
SoundRep        3448
WordRep         2752
dtype: int64


In [ ]:
def extract_wav2vec_sequences(dataframe, desc="Extracting", augment=False):
    processor = Wav2Vec2Processor.from_pretrained(MODEL_NAME)
    model = Wav2Vec2Model.from_pretrained(MODEL_NAME).to(DEVICE)
    model.eval()
    augmenter = Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.5)
    ])
    
    # Pre-calculate the final size of the dataset 
    final_size = 0
    if augment:
        for index, row in tqdm(dataframe.iterrows(), total=len(dataframe), desc="Calculating final size"):
            final_size += 1   
            
            # CUSTOM AUGMENTATION LOGIC 
            num_augmentations = 0
            if row['Block'] == 0:
                if row['WordRep'] == 1 or row['SoundRep'] == 1:
                    num_augmentations = 3
                elif row['Prolongation'] == 1:
                    num_augmentations = 1
            elif row['Block'] == 1:
                if row['SoundRep'] == 1:
                    num_augmentations = 1
                elif row['WordRep'] == 1:
                    num_augmentations = 1
            
            if row[LABEL_COLS].sum() == 1 and row['WordRep'] == 1:
                num_augmentations += 3
            
            if row[LABEL_COLS].sum() == 2 and row['WordRep'] == 1 and row['SoundRep'] == 1:
                num_augmentations += 2
                
            if row[LABEL_COLS].sum() == 1 and row['SoundRep'] == 1:
                num_augmentations += 2
            
            final_size += num_augmentations
    else:
        final_size = len(dataframe)
    
    print(f"Final dataset size for '{desc}' will be {final_size} samples.")

    # Pre-allocate NumPy arrays 
    all_sequences = np.zeros((final_size, MAX_SEQ_LEN, EMBEDDING_DIM), dtype=np.float32)
    all_labels = np.zeros((final_size, len(ALL_LABELS)), dtype=np.int8)
    
    current_idx = 0

    with torch.no_grad():
        for i in tqdm(range(0, len(dataframe), BATCH_SIZE), desc=f"{desc} Batches"):
            batch_df = dataframe.iloc[i:i+BATCH_SIZE]
            
            audio_batch = []
            labels_batch = []
            
            for index, row in batch_df.iterrows():
                clip_path = str(CLIPS_DIR / f"{row['Clip']}.wav")
                audio, sr = librosa.load(clip_path, sr=SAMPLING_RATE)
                current_labels = row[ALL_LABELS].values
                
                audio_batch.append(audio)
                labels_batch.append(current_labels)

                if augment:
                    # CUSTOM AUGMENTATION LOGIC 
                    num_augmentations = 0
                    if row['Block'] == 0:
                        if row['WordRep'] == 1 or row['SoundRep'] == 1:
                            num_augmentations = 3
                        elif row['Prolongation'] == 1:
                            num_augmentations = 1
                    elif row['Block'] == 1:
                        if row['SoundRep'] == 1:
                            num_augmentations = 1
                        elif row['WordRep'] == 1:
                            num_augmentations = 1

                    if row[LABEL_COLS].sum() == 1 and row['WordRep'] == 1:
                        num_augmentations += 3
                    
                    if row[LABEL_COLS].sum() == 2 and row['WordRep'] == 1 and row['SoundRep'] == 1:
                        num_augmentations += 2

                    if row[LABEL_COLS].sum() == 1 and row['SoundRep'] == 1:
                        num_augmentations += 2

                    for _ in range(num_augmentations):
                        augmented_audio = augmenter(samples=audio, sample_rate=SAMPLING_RATE)
                        audio_batch.append(augmented_audio)
                        labels_batch.append(current_labels)

            inputs = processor(audio_batch, sampling_rate=SAMPLING_RATE, return_tensors="pt", padding=True)
            inputs = inputs.to(DEVICE)
            sequences = model(**inputs).last_hidden_state.cpu().numpy()
            
            # Fill the pre-allocated arrays
            for j, seq in enumerate(sequences):
                if current_idx >= final_size: break
                if seq.shape[0] < MAX_SEQ_LEN:
                    pad_width = MAX_SEQ_LEN - seq.shape[0]
                    seq = np.pad(seq, ((0, pad_width), (0, 0)), mode='constant')
                else:
                    seq = seq[:MAX_SEQ_LEN, :]
                
                all_sequences[current_idx] = seq
                all_labels[current_idx] = labels_batch[j]
                current_idx += 1
            if current_idx >= final_size: break

    return all_sequences, all_labels

print("\n--- Starting Wav2Vec2 sequence extraction... ---")
X_train, y_train = extract_wav2vec_sequences(train_df, desc="Training", augment=True)
np.savez_compressed(TRAIN_DATA_PATH, x=X_train, y=y_train)
print(f"Saved compressed training data with shape: {X_train.shape}")

# distribution AFTER augmentation 
final_train_labels_df = pd.DataFrame(y_train, columns=ALL_LABELS)
print("\n--- Final Label Distribution in Training Set (After Augmentation) ---")
print(final_train_labels_df.sum().sort_values(ascending=False))

# Clear memory before processing the next set for memory-constrained environments 
del X_train, y_train

X_val, y_val = extract_wav2vec_sequences(val_df, desc="Validation", augment=False)
np.savez_compressed(VAL_DATA_PATH, x=X_val, y=y_val)
print(f"Saved compressed validation data with shape: {X_val.shape}")
del X_val, y_val

X_test, y_test = extract_wav2vec_sequences(test_df, desc="Test", augment=False)
np.savez_compressed(TEST_DATA_PATH, x=X_test, y=y_test)
print(f"Saved compressed test data with shape: {X_test.shape}")
del X_test, y_test

print(f"\n--- Feature extraction complete. All files saved in '{OUTPUT_DIR}' directory. ---")


--- Starting Wav2Vec2 sequence extraction... ---


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/163 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.json:   0%|          | 0.00/291 [00:00<?, ?B/s]

...

Saved compressed validation data with shape: (734, 150, 768)


Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Final dataset size for 'Test' will be 8622 samples.


Test Batches:   0%|          | 0/270 [00:00<?, ?it/s]

Saved compressed test data with shape: (8622, 150, 768)

--- Feature extraction complete. All files saved in 'output_wav2vec_custom_augmented' directory. ---
